## Emission maps

In [ ]:
import numpy as np
import yt
import unyt
from yt import YTArray
from yt.data_objects.level_sets.api import Clump, find_clumps
import argparse
import os
from astropy.table import Table
from astropy.io import ascii
import multiprocessing as multi
import datetime
from scipy import interpolate
import shutil
import matplotlib.pyplot as plt
import cmasher as cmr
import matplotlib.colors as mcolors
import h5py

# These imports are FOGGIE-specific files
from foggie.utils.consistency import *
from foggie.utils.get_run_loc_etc import get_run_loc_etc
from foggie.utils.yt_fields import *
from foggie.utils.foggie_load import *
from foggie.utils.analysis_utils import *

# These imports for datashader plots
import datashader as dshader
from datashader.utils import export_image
import datashader.transfer_functions as tf
import pandas as pd
import matplotlib as mpl
import numpy as np


def generate_foggie_paths(halo, run, snap):
    # Define base paths
    foggie_base_dir = "/Volumes/FoggieVida/" 
    code_base_path = "/Users/vidasaeedzadeh/Projects/foggie/foggie/"
    output_base_dir = "/Users/vidasaeedzadeh/Projects/foggie_outputs/"

    # Zero-pad the halo number to 6 digits
    halo_number = halo.zfill(6)

    # Define directory and file paths dynamically
    foggie_dir = os.path.join(foggie_base_dir, f"halo_{halo_number}", run + '/')
    snap_name = os.path.join(foggie_dir, snap, snap)
    halo_c_v_name = os.path.join(code_base_path, f"halo_infos/{halo_number}/{run}/halo_c_v")
    trackname = os.path.join(code_base_path, f"halo_tracks/{halo_number}/nref11n_selfshield_15/halo_track_200kpc_nref9")

    # Output directory (adjust based on needs)
    output_dir = output_base_dir

    # Return paths
    return foggie_dir,code_base_path, snap_name, halo_c_v_name, trackname, output_dir

# specify halo and snapshot
halo = '8508'
run = 'ludicrous/nref13c_nref9f.enhance'
snap = 'DD2509'

foggie_dir,code_path, snap_name, halo_c_v_name, trackname, output_dir = generate_foggie_paths(halo, run, snap)



# System and plotting settings
system = ''  # System you're using
plot = 'emission_FRB'  # Options: emission_map, emission_map_vbins, or emission_FRB
ions = ['CIV', 'OVI']#['Lyalpha', 'Halpha', 'CIII','SiII','SiIII','SiIV','MgII']#['Lyalpha', 'Halpha', 'CIII', 'CIV', 'OVI','SiII','SiIII','SiIV','MgII']  
Dragonfly_limit = False
Aspera_limit = False
save_suffix = ""
file_suffix = ""
segmentation_filter='radial_velocity' # it can also be 'metallicity'

# Filtering settings (optional)
filter_type = None  # Type of filter, e.g., 'temperature', 'density'
filter_value = None  # Value for the filter, e.g., 1e4 for temperature < 1e4 K


def scale_by_metallicity(values,assumed_Z,wanted_Z):
    # The Cloudy calculations assumed a single metallicity (typically solar).
    # This function scales the emission by the metallicity of the gas itself to
    # account for this discrepancy.
    wanted_ratio = (10.**(wanted_Z))/(10.**(assumed_Z))
    return values*wanted_ratio

def make_Cloudy_table(table_index):
    # This function takes all of the Cloudy files and compiles them into one table
    # for use in the emission functions
    # table_index is the column in the Cloudy output files that is being read.
    # each table_index value corresponds to a different emission line

    # this is the the range and number of bins for which Cloudy was run
    # i.e. the temperature and hydrogen number densities gridded in the
    # Cloudy run. They must match or the table will be incorrect.
    hden_n_bins, hden_min, hden_max = 15, -5, 2 #17, -6, 2 #23, -9, 2
    T_n_bins, T_min, T_max = 51, 3, 8 #71, 2, 8

    hden=np.linspace(hden_min,hden_max,hden_n_bins)
    T=np.linspace(T_min,T_max, T_n_bins)
    table = np.zeros((hden_n_bins,T_n_bins))
    for i in range(hden_n_bins):
            table[i,:]=[float(l.split()[table_index]) for l in open(cloudy_path%(i+1)) if l[0] != "#"]
    return hden,T,table

def make_Cloudy_table_thin(table_index):
    hden_n_bins, hden_min, hden_max = 17, -5, 2
    T_n_bins, T_min, T_max = 51, 3, 8 #71, 2, 8

    hden=np.linspace(hden_min,hden_max,hden_n_bins)
    T=np.linspace(T_min,T_max, T_n_bins)
    table = np.zeros((hden_n_bins,T_n_bins))
    for i in range(hden_n_bins):
            table[i,:]=[float(l.split()[table_index]) for l in open(cloudy_path_thin%(i+1)) if l[0] != "#"]
    return hden,T,table

def _Emission_LyAlpha(field,data):
    H_N=np.log10(np.array(data["H_nuclei_density"]))
    Temperature=np.log10(np.array(data["Temperature"]))
    dia1 = bl_LA(H_N,Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line = (10**dia1)*((10.0**H_N)**2.0)
    emission_line = emission_line/(4.*np.pi*1.63e-11)
    return emission_line*ytEmU

def _Emission_HAlpha_ALTunits(field,data):
    H_N = np.log10(np.array(data['H_nuclei_density']))
    Temperature = np.log10(np.array(data['Temperature']))
    dia1 = bl_HA(H_N,Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line=(10.**dia1)*((10.**H_N)**2.0)
    emission_line = emission_line/(4.*np.pi)
    emission_line = emission_line/4.25e10 # convert steradian to arcsec**2
    return emission_line*ytEmUALT

def _Emission_HAlpha(field,data):
    H_N = np.log10(np.array(data['H_nuclei_density']))
    Temperature = np.log10(np.array(data['Temperature']))
    dia1 = bl_HA(H_N,Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line=(10.**dia1)*((10.**H_N)**2.0)
    emission_line = emission_line/(4.*np.pi*3.03e-12)
    return emission_line*ytEmU

def _Emission_CIII_977(field,data):
    H_N=np.log10(np.array(data["H_nuclei_density"]))
    Temperature=np.log10(np.array(data["Temperature"]))
    dia1 = bl_CIII_977(H_N,Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    emission_line=(10.0**dia1)*((10.0**H_N)**2.0)
    emission_line = emission_line/(4.*np.pi*2.03e-11)
    emission_line = scale_by_metallicity(emission_line,0.0,np.log10(np.array(data['metallicity'])))
    return emission_line*ytEmU

def _Emission_CIV_1548(field,data):
        H_N=np.log10(np.array(data["H_nuclei_density"]))
        Temperature=np.log10(np.array(data["Temperature"]))
        dia1 = bl_CIV_1(H_N,Temperature)
        idx = np.isnan(dia1)
        dia1[idx] = -200.
        emission_line=(10.0**dia1)*((10.0**H_N)**2.0)
        emission_line = emission_line/(4.*np.pi*1.28e-11)
        emission_line = scale_by_metallicity(emission_line,0.0,np.log10(np.array(data['metallicity'])))
        return emission_line*ytEmU

def _Emission_OVI(field,data):
    H_N=np.log10(np.array(data["H_nuclei_density"]))
    Temperature=np.log10(np.array(data["Temperature"]))
    dia1 = bl_OVI_1(H_N,Temperature)
    dia2 = bl_OVI_2(H_N,Temperature)
    idx = np.isnan(dia1)
    dia1[idx] = -200.
    dia2[idx] = -200.
    emission_line=((10.0**dia1)+(10**dia2))*((10.0**H_N)**2.0)
    emission_line = emission_line/(4.*np.pi*1.92e-11)
    emission_line = scale_by_metallicity(emission_line,0.0,np.log10(np.array(data['metallicity'])))
    return emission_line * ytEmU

def _Emission_SiIII_1207(field,data):
        H_N=np.log10(np.array(data["H_nuclei_density"]))
        Temperature=np.log10(np.array(data["Temperature"]))
        dia1 = bl_SiIII_1207(H_N,Temperature)
        idx = np.isnan(dia1)
        dia1[idx] = -200.
        emission_line=(10.0**dia1)*((10.0**H_N)**2.0)
        emission_line = emission_line/(4.*np.pi*1.65e-11)
        emission_line = scale_by_metallicity(emission_line,0.0,np.log10(np.array(data['metallicity'])))
        return emission_line*ytEmU

def _Emission_SiII_1814(field,data):
        H_N=np.log10(np.array(data["H_nuclei_density"]))
        Temperature=np.log10(np.array(data["Temperature"]))
        dia1 = bl_SiII_1814(H_N,Temperature)
        idx = np.isnan(dia1)
        dia1[idx] = -200.
        emission_line=(10.0**dia1)*((10.0**H_N)**2.0)
        emission_line = emission_line/(4.*np.pi*1.65e-11)
        emission_line = scale_by_metallicity(emission_line,0.0,np.log10(np.array(data['metallicity'])))
        return emission_line*ytEmU

def _Emission_SiIV_1394(field,data):
        H_N=np.log10(np.array(data["H_nuclei_density"]))
        Temperature=np.log10(np.array(data["Temperature"]))
        dia1 = bl_SiIV_1394(H_N,Temperature)
        idx = np.isnan(dia1)
        dia1[idx] = -200.
        emission_line=(10.0**dia1)*((10.0**H_N)**2.0)
        emission_line = emission_line/(4.*np.pi*1.65e-11)
        emission_line = scale_by_metallicity(emission_line,0.0,np.log10(np.array(data['metallicity'])))
        return emission_line*ytEmU

def _Emission_MgII_2796(field,data):
        H_N=np.log10(np.array(data["H_nuclei_density"]))
        Temperature=np.log10(np.array(data["Temperature"]))
        dia1 = bl_MgII_2796(H_N,Temperature)
        idx = np.isnan(dia1)
        dia1[idx] = -200.
        emission_line=(10.0**dia1)*((10.0**H_N)**2.0)
        emission_line = emission_line/(4.*np.pi*1.65e-11)
        emission_line = scale_by_metallicity(emission_line,0.0,np.log10(np.array(data['metallicity'])))
        return emission_line*ytEmU

In [5]:
def filter_ds(box):
    '''This function filters the yt data object passed in as 'box' into inflow and outflow regions,
    based on metallicity, and returns the box filtered into these regions.'''

    if (segmentation_filter=='metallicity'):
        box_inflow = box.include_below(('gas','metallicity'), 0.01, 'Zsun')
        box_outflow = box.include_above(('gas','metallicity'), 1., 'Zsun')
        box_neither = box.include_above(('gas','metallicity'), 0.01, 'Zsun')
        box_neither = box_neither.include_below(('gas','metallicity'), 1., 'Zsun')
    elif (segmentation_filter=='radial_velocity'):
        box_inflow = box.include_below(('gas','radial_velocity_corrected'), -100., 'km/s')
        box_outflow = box.include_above(('gas','radial_velocity_corrected'), 200., 'km/s')
        box_neither = box.include_above(('gas','radial_velocity_corrected'), -100., 'km/s')
        box_neither = box_neither.include_below(('gas','radial_velocity_corrected'), 200., 'km/s')

    return box_inflow, box_outflow, box_neither

In [6]:
def filter_ds_disk_cgm(ds):
    '''Filters the yt data object into disk and CGM regions based on density clumping and returns them as box_disk and box_cgm.'''

    # Determine the density cut factor based on the simulation's current time
    current_time = ds.current_time.in_units('Myr').v
    if current_time <= 8656.88:
        density_cut_factor = 1.0
    elif current_time <= 10787.12:
        density_cut_factor = 1.0 - 0.9 * (current_time - 8656.88) / 2130.24
    else:
        density_cut_factor = 0.1

    # Create a spherical region around the halo center with a 60 kpc radius
    sphere = ds.sphere(center=ds.halo_center_kpc, radius=(60, 'kpc'))
    # Separate the sphere into CGM and ISM (disk) regions based on density
    sph_cgm = sphere.cut_region([f"obj['density'] < {density_cut_factor * cgm_density_max}"])
    sph_ism = sphere.cut_region([f"obj['density'] > {density_cut_factor * cgm_density_max}"])

    # Identify the disk by finding the densest clumps within the ISM region
    master_clump = Clump(sphere, ("gas", "density"))
    master_clump.add_validator("min_cells", 200)
    c_min = 2 * sph_ism["gas", "density"].min()
    c_max = sph_ism["gas", "density"].max()
    step = c_max / c_min
    find_clumps(master_clump, c_min, c_max, step)

    # Extract the clump that represents the disk
    leaf_clumps = master_clump.leaves
    disk_clump = leaf_clumps[1]  # Assuming the second clump is the disk

    # Mask cells belonging to the disk
    clump_indices = np.array(disk_clump["index", "cell_id"])
    
    def _masked_field(field, data):
        return ~np.isin(data["index", "cell_id"], clump_indices)

    ds.add_field(
        ("index", "masked_field"),
        function=_masked_field,
        sampling_type="cell",
        units="",
        force_override=True
    )

    # Separate disk and CGM regions based on the mask
    box_disk = disk_clump.data
    box_cgm = sphere.cut_region(["obj['index','masked_field']"])

    return box_disk, box_cgm


In [7]:
def make_FRB(ds, refine_box, snap, ions, filter_type=None, filter_value=None):
    '''This function takes the dataset 'ds' and the refine box region 'refine_box' and
    makes a fixed resolution buffer of surface brightness from edge-on, face-on,
    and arbitrary orientation projections of all ions in the list 'ions'.'''

    halo_name = halo_dict[str(halo)]

    # Determine the resolution based on pixel size
    pix_res = float(np.min(refine_box['dx'].in_units('kpc')))
    #res = int(ds.refine_width / pix_res)
    res = int(100 / pix_res)
    print('z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1))

    # Apply inflow/outflow or disk/CGM filtering using the filter_ds function if specified
    if filter_type == 'inflow_outflow':
        # Apply the inflow/outflow filtering
        box_inflow, box_outflow, box_neither = filter_ds(ds.all_data())
        data_sources = {'inflow': box_inflow, 'outflow': box_outflow, 'neither': box_neither}

    elif filter_type == 'disk_cgm':
        # Apply the disk/CGM filtering
        box_disk, box_cgm = filter_ds_disk_cgm(ds)
        data_sources = {'disk': box_disk, 'cgm': box_cgm}
        
    else:
        # Standard filtering or no filter
        data_sources = {'all': ds.all_data()}
        if filter_type and filter_value:
            if filter_type == 'temperature':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
            elif filter_type == 'density':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'density'] > {filter_value})"])
            else:
                raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

    # Create HDF5 file for saving emission maps
    save_path = prefix + 'FRBs/'
    os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
    f = h5py.File( save_path + halo_name + '_emission_maps' + save_suffix + '.hdf5', 'a')
    grp = f.create_group('z=%.1f' % ds.get_parameter('CosmologyCurrentRedshift', 1))
    grp.attrs.create("image_extent_kpc", ds.refine_width)
    grp.attrs.create("redshift", ds.get_parameter('CosmologyCurrentRedshift'))
    grp.attrs.create("halo_name", halo_name)
    grp.attrs.create("emission_units", 'photons/sec/cm^2/sr')
    grp.attrs.create("gas_density_units", 'g/cm^2')
    grp.attrs.create("stars_density_units", 'Msun/kpc^2')

    # Loop through ions and create projections for each region
    for region, data_source in data_sources.items():
        for ion in ions:
            print(ion)

            #Edge-on projection
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, ('gas', 'Emission_' + ions_dict[ion]),
                                          center=ds.halo_center_kpc, data_source=data_source,width=(100, 'kpc'),
                                          north_vector=ds.z_unit_disk, buff_size=[res, res])#width=(ds.refine_width, 'kpc'),
            frb_edge = proj_edge.frb[('gas', 'Emission_' + ions_dict[ion])]
            dset1 = grp.create_dataset(f"{ion}_emission_edge_{region}", data=frb_edge)

            # Set colormap and save projection plot
            mymap = cmr.get_sub_cmap('cmr.flamingo', 0.2, 0.8)
            mymap.set_bad("#421D0F")
            proj_edge.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_edge.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_edge.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission [photons s$^{-1}$ cm$^{-2}$ sr$^{-1}$]')
            proj_edge.set_font_size(20)
            proj_edge.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            save_path = prefix + 'FRBs/'
            os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
            proj_edge.save(save_path + f'{snap}_{ion}_emission_map_edge-on_{region}' + save_suffix + '.png')

            # Face-on projection
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, ('gas', 'Emission_' + ions_dict[ion]),
                                          center=ds.halo_center_kpc, data_source=data_source,width=(100, 'kpc'),
                                          north_vector=ds.x_unit_disk, buff_size=[res, res])#width=(ds.refine_width, 'kpc'),
            frb_face = proj_face.frb[('gas', 'Emission_' + ions_dict[ion])]
            dset2 = grp.create_dataset(f"{ion}_emission_face_{region}", data=frb_face)

            # Set colormap and save projection plot
            proj_face.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_face.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_face.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission [photons s$^{-1}$ cm$^{-2}$ sr$^{-1}$]')
            proj_face.set_font_size(20)
            proj_face.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            save_path = prefix + 'FRBs/'
            os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
            proj_face.save(save_path + f'{snap}_{ion}_emission_map_face-on_{region}' + save_suffix + '.png')

    # Close the HDF5 file after saving the datasets
    print('finished')
    f.close()


In [8]:
def emission_map_vbins(ds, snap, ions, filter_type=None, filter_value=None):
    '''Makes many emission maps for each ion in 'ions', oriented both edge-on and face-on, for each line-of-sight velocity bin.'''

    vbins = np.arange(-500., 550., 50.)  # Velocity bins
    ad = ds.all_data()

    for i in range(len(ions)):
        ion = ions[i]

        # Choose colormap based on ion and emission limits
        if (ion == 'Halpha') and Dragonfly_limit:
            cmap1 = cmr.take_cmap_colors('cmr.flamingo', 9, cmap_range=(0.4, 0.8), return_fmt='rgba')
            cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 3, cmap_range=(0.2, 0.6), return_fmt='rgba')
            cmap = np.hstack([cmap2, cmap1])
            mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
        elif (ion == 'OVI') and Aspera_limit:
            cmap1 = cmr.take_cmap_colors('cmr.flamingo', 4, cmap_range=(0.4, 0.8), return_fmt='rgba')
            cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 6, cmap_range=(0.2, 0.6), return_fmt='rgba')
            cmap = np.hstack([cmap2, cmap1])
            mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
        else:
            mymap = cmr.get_sub_cmap('cmr.flamingo', 0.2, 0.8)
        mymap.set_bad(mymap.colors[0])

        # Loop through each velocity bin
        for v in range(len(vbins) - 1):
            # Filter the data by the current velocity bin
            vbox = ds.cut_region(ad, [f"obj[('gas', 'vx_disk')] > {vbins[v]:.1f}"])
            vbox = ds.cut_region(vbox, [f"obj[('gas', 'vx_disk')] < {vbins[v+1]:.1f}"])

            # Apply filtering if specified (e.g., temperature or density cut)
            if filter_type and filter_value:
                if filter_type == 'temperature':
                    vbox = vbox.cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
                elif filter_type == 'density':
                    vbox = vbox.cut_region([f"(obj['gas', 'density'] > {filter_value})"])
                else:
                    raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

            # Edge-on projection
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), 
                                          center=ds.halo_center_kpc, width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.z_unit_disk, data_source=vbox)
            proj_edge.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_edge.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_edge.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission [photons s$^{-1}$ cm$^{-2}$ sr$^{-2}$]')
            proj_edge.set_font_size(20)
            proj_edge.annotate_title(f'$%d < v_{{\\rm los}} < %d$' % (vbins[v], vbins[v+1]))
            proj_edge.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_edge.save(prefix + 'EmissionMap/' + snap + '_' + ion + '_emission_map_edge-on_vbin' + str(v) + save_suffix + '.png')

            # Face-on projection
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), 
                                          center=ds.halo_center_kpc, width=(ds.refine_width, 'kpc'),
                                          north_vector=ds.x_unit_disk, data_source=vbox)
            proj_face.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_face.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_face.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission [photons s$^{-1}$ cm$^{-2}$ sr$^{-2}$]')
            proj_face.set_font_size(20)
            proj_face.annotate_title(f'$%d < v_{{\\rm los}} < %d$' % (vbins[v], vbins[v+1]))
            proj_face.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_face.save(prefix + 'EmissionMap/' + snap + '_' + ion + '_emission_map_face-on_vbin' + str(v) + save_suffix + '.png')


In [9]:
def emission_map(ds, snap, ions, filter_type=None, filter_value=None):
    '''Makes emission maps for each ion in 'ions', oriented both edge-on and face-on, with optional filtering and inflow/outflow support.'''

    print(f'Called emission_map function with ions: {ions}')

    # Apply inflow/outflow or disk/CGM filtering using the filter_ds function if specified
    if filter_type == 'inflow_outflow':
        # Apply the inflow/outflow filtering
        box_inflow, box_outflow, box_neither = filter_ds(ds.all_data())
        data_sources = {'inflow': box_inflow, 'outflow': box_outflow, 'neither': box_neither}

    elif filter_type == 'disk_cgm':
        # Apply the disk/CGM filtering
        box_disk, box_cgm = filter_ds_disk_cgm(ds)
        data_sources = {'disk': box_disk, 'cgm': box_cgm}
        
    else:
        # Standard filtering or no filter
        data_sources = {'all': ds.all_data()}
        if filter_type and filter_value:
            if filter_type == 'temperature':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'temperature'] < {filter_value})"])
            elif filter_type == 'density':
                data_sources['all'] = data_sources['all'].cut_region([f"(obj['gas', 'density'] > {filter_value})"])
            else:
                raise ValueError("Unsupported filter type. Supported types: 'temperature', 'density'.")

    # Generate a filter descriptor for use in file names
    if filter_type and filter_value:
        filter_desc = f"{filter_type}_{filter_value}"
    elif filter_type:
        filter_desc = f"{filter_type}"  
    else:
        filter_desc = 'nofilter'

    for region, data_source in data_sources.items():
        print('data_source:', data_source)
        for ion in ions:
            # Choose colormap based on the ion and any special limits
            if (ion == 'Halpha') and Dragonfly_limit:
                cmap1 = cmr.take_cmap_colors('cmr.flamingo', 9, cmap_range=(0.4, 0.8), return_fmt='rgba')
                cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 3, cmap_range=(0.2, 0.6), return_fmt='rgba')
                cmap = np.hstack([cmap2, cmap1])
                mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
            elif (ion == 'OVI') and Aspera_limit:
                cmap1 = cmr.take_cmap_colors('cmr.flamingo', 4, cmap_range=(0.4, 0.8), return_fmt='rgba')
                cmap2 = cmr.take_cmap_colors('cmr.neutral_r', 6, cmap_range=(0.2, 0.6), return_fmt='rgba')
                cmap = np.hstack([cmap2, cmap1])
                mymap = mcolors.LinearSegmentedColormap.from_list('cmap', cmap)
            else:
                mymap = cmr.get_sub_cmap('cmr.flamingo', 0.2, 0.8)

            # Edge-on projection
            proj_edge = yt.ProjectionPlot(ds, ds.x_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), center=ds.halo_center_kpc, width=(100, 'kpc'), north_vector=ds.z_unit_disk, data_source=data_source)
            proj_edge.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_edge.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_edge.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission [photons s$^{-1}$ cm$^{-2}$ sr$^{-2}$]')
            proj_edge.set_font_size(20)
            proj_edge.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_edge.annotate_title(f"{region} Edge-on")
            save_path = prefix + f'EmissionMap/{filter_desc}/'
            os.makedirs(save_path, exist_ok=True)
            proj_edge.save(save_path + f'{snap}_{ion}_emission_map_edge_on_100kpc_{region}' + save_suffix + '.png')

            # Face-on projection
            proj_face = yt.ProjectionPlot(ds, ds.z_unit_disk, ('gas', 'Emission_' + ions_dict[ion]), center=ds.halo_center_kpc, width=(100, 'kpc'), north_vector=ds.x_unit_disk, data_source=data_source)
            proj_face.set_cmap('Emission_' + ions_dict[ion], mymap)
            proj_face.set_zlim('Emission_' + ions_dict[ion], zlim_dict[ion][0], zlim_dict[ion][1])
            proj_face.set_colorbar_label('Emission_' + ions_dict[ion], label_dict[ion] + ' Emission [photons s$^{-1}$ cm$^{-2}$ sr$^{-2}$]')
            proj_face.set_font_size(20)
            proj_face.annotate_timestamp(corner='upper_left', redshift=True, time=True, draw_inset_box=True)
            proj_face.annotate_title(f"{region} Face-on")
            proj_face.save(save_path + f'{snap}_{ion}_emission_map_face_on_100kpc_{region}' + save_suffix + '.png')


In [10]:
def load_and_calculate(snap, ions, filter_type=None, filter_value=None):
    '''Loads the simulation snapshot and makes the requested plots, with optional filtering.'''

    # # Load simulation output
    # if system == 'pleiades_cassi':
    #     print('Copying directory to /tmp')
    #     snap_dir = '/nobackup/clochhaa/tmp/' + halo + '/' + run + '/' + target_dir + '/' + snap
    #     os.makedirs(snap_dir)
    #     snap_name = foggie_dir + run_dir + snap + '/' + snap
    # else:
    #     snap_name = foggie_dir + snap + '/' + snap
    
    # ds, refine_box = foggie_load(snap_name, trackname, do_filter_particles=True, halo_c_v_name=halo_c_v_name, disk_relative=True, correct_bulk_velocity=True)#, smooth_AM_name=smooth_AM_name)
    # zsnap = ds.get_parameter('CosmologyCurrentRedshift')

    # Generate emission maps based on the plot type
    if 'emission_map' in plot:
        if 'vbins' not in plot:
            # Call emission map function with optional filtering
            emission_map(ds, snap, ions, filter_type=filter_type, filter_value=filter_value)
        else:
            # Call velocity-binned emission map function with optional filtering
            emission_map_vbins(ds, snap, ions, filter_type=filter_type, filter_value=filter_value)
    
    if 'emission_FRB' in plot:
        # Call the FRB function with optional filtering
        make_FRB(ds, refine_box, snap, ions, filter_type=filter_type, filter_value=filter_value)

    

if __name__ == "__main__":


    # if ('feedback' in run) and ('track' in run):
    #     foggie_dir = '/nobackup/jtumlins/halo_008508/feedback-track/'
    #     run_dir = run + '/'
    
    # Set directory for output location, making it if necessary
    prefix = output_dir + 'ions_halo_00' + halo + '/' + run + '/'
    if not (os.path.exists(prefix)): os.system('mkdir -p ' + prefix)
    table_loc = prefix + 'Tables/'

    print('foggie_dir: ', foggie_dir)
    catalog_dir = code_path + 'halo_infos/00' + halo + '/' + run + '/'
    halo_c_v_name = catalog_dir + 'halo_c_v'
    #smooth_AM_name = catalog_dir + 'AM_direction_smoothed'

    cloudy_path = code_path + "emission/cloudy_z0_selfshield/sh_z0_HM12_run%i.dat"
    # These are the typical units that Lauren uses
    # NOTE: This is a volumetric unit since it's for the emissivity of each cell
    # Emission / surface brightness comes from the projections
    emission_units = 's**-1 * cm**-3 * steradian**-1'
    ytEmU = unyt.second**-1 * unyt.cm**-3 * unyt.steradian**-1

    # These are a second set of units that a lot of observers prefer
    # NOTE: This is a volumetric unit since it's for the emissivity of each cell
    # Emission / surface brightness comes from the projections
    emission_units_ALT = 'erg * s**-1 * cm**-3 * arcsec**-2'
    ytEmUALT = unyt.erg * unyt.second**-1 * unyt.cm**-3 * unyt.arcsec**-2

    ####################################
    ## BEGIN CREATING EMISSION FIELDS ##
    ####################################

    # To make the emissivity fields, you need to follow a number of steps
    # 1. Read in the Cloudy values for a given emission line
    # 2. Create the n_H and T grids that represent the desired range of values
    # 3. Set up interpolation function for the emissivity values across the grids
    #    so the code can use the n_H and T values of a simulation grid cell to
    #    interpolate the correct emissivity value
    # 4. Define the emission field for the line
    # 5. Add the line as a value in yt

    ############################
    ## H-alpha ##
    ############################
    # 1. Read cloudy file
    hden_pts,T_pts,table_HA = make_Cloudy_table(2)
    # 2. Create grids (only need to do this for first emission line)
    hden_pts,T_pts = np.meshgrid(hden_pts,T_pts)
    pts = np.array((hden_pts.ravel(),T_pts.ravel())).T

    #set up interpolation fundtion
    sr_HA = table_HA.T.ravel()
    bl_HA = interpolate.LinearNDInterpolator(pts,sr_HA)
    # 5. Add field
    yt.add_field(('gas','Emission_HAlpha'),units=emission_units,function=_Emission_HAlpha,take_log=True,force_override=True,sampling_type='cell')

    ############################
    ## Ly-alpha ##
    ############################
    hden_pts,T_pts,table_LA = make_Cloudy_table(1)
    sr_LA = table_LA.T.ravel()
    bl_LA = interpolate.LinearNDInterpolator(pts,sr_LA)
    yt.add_field(('gas','Emission_LyAlpha'),units=emission_units,function=_Emission_LyAlpha,take_log=True,force_override=True,sampling_type='cell')

    ############################
    ## CIII 977 ##
    ############################
    hden1,T1,table_CIII_977 = make_Cloudy_table(7)
    sr_CIII_977 = table_CIII_977.T.ravel()
    bl_CIII_977 = interpolate.LinearNDInterpolator(pts,sr_CIII_977)
    yt.add_field(("gas","Emission_CIII_977"),units=emission_units,function=_Emission_CIII_977,take_log=True,force_override=True,sampling_type='cell')

    ############################
    ## CIV 1548 ##
    ############################
    hden1, T1, table_CIV_1 = make_Cloudy_table(3)
    sr_CIV_1 = table_CIV_1.T.ravel()
    bl_CIV_1 = interpolate.LinearNDInterpolator(pts,sr_CIV_1)
    yt.add_field(("gas","Emission_CIV_1548"),units=emission_units,function=_Emission_CIV_1548,take_log=True,force_override=True,sampling_type='cell')

    ############################
    ## O VI both 1032 and 1037 ##
    ############################
    hden1, T1, table_OVI_1 = make_Cloudy_table(5)
    hden1, T1, table_OVI_2 = make_Cloudy_table(6)
    sr_OVI_1 = table_OVI_1.T.ravel()
    sr_OVI_2 = table_OVI_2.T.ravel()
    bl_OVI_1 = interpolate.LinearNDInterpolator(pts,sr_OVI_1)
    bl_OVI_2 = interpolate.LinearNDInterpolator(pts,sr_OVI_2)
    yt.add_field(("gas","Emission_OVI"),units=emission_units,function=_Emission_OVI,take_log=True,force_override=True,sampling_type='cell')

    ############################
    ## Si III 1207 ##
    ############################
    # This one in optically thin Cloudy table, so need to load new table
    cloudy_path_thin = code_path + "emission/cloudy_z0_HM05/bertone_run%i.dat"
    hden_pts,T_pts,table_SiIII_1207 = make_Cloudy_table_thin(11)
    hden_pts,T_pts = np.meshgrid(hden_pts,T_pts)
    pts = np.array((hden_pts.ravel(),T_pts.ravel())).T
    sr_SiIII_1207 = table_SiIII_1207.T.ravel()
    bl_SiIII_1207 = interpolate.LinearNDInterpolator(pts,sr_SiIII_1207)
    yt.add_field(('gas','Emission_SiIII_1207'),units=emission_units,function=_Emission_SiIII_1207,take_log=True,force_override=True,sampling_type='cell')

    ions_dict = {'Lyalpha':'LyAlpha', 'Halpha':'HAlpha', 'CIII':'CIII_977',
                 'CIV':'CIV_1548','OVI':'OVI', 'SiIII':'SiIII_1207'}
    label_dict = {'Lyalpha':r'Ly-$\alpha$', 'Halpha':r'H$\alpha$', 'CIII':'C III',
                'CIV':'C IV','OVI':'O VI', 'SiIII':'Si III'}
    zlim_dict = {'Lyalpha':[1e-1,1e7], 'Halpha':[1e-1,1e6], 'CIII':[1e-4,1e0],
                 'CIV':[1e-2,1e4], 'OVI':[1e-2,1e5], 'SiIII':[1e-1,1e4]}

foggie_dir:  /Volumes/FoggieVida/halo_008508/ludicrous/nref13c_nref9f.enhance/


In [11]:
# Load simulation output
snap_name = foggie_dir + snap + '/' + snap

ds, refine_box = foggie_load(snap_name, trackname, do_filter_particles=True, halo_c_v_name=halo_c_v_name, disk_relative=True, correct_bulk_velocity=True)#, smooth_AM_name=smooth_AM_name)
zsnap = ds.get_parameter('CosmologyCurrentRedshift')

yt : [INFO     ] 2024-11-15 09:12:48,387 Parameters: current_time              = 638.06651531954
yt : [INFO     ] 2024-11-15 09:12:48,388 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2024-11-15 09:12:48,388 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-11-15 09:12:48,388 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2024-11-15 09:12:48,389 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2024-11-15 09:12:48,389 Parameters: current_redshift          = 0.0021037994044062
yt : [INFO     ] 2024-11-15 09:12:48,389 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2024-11-15 09:12:48,389 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2024-11-15 09:12:48,389 Parameters: omega_radiation           = 0
yt : [INFO     ] 2024-11-15 09:12:48,389 Parameters: hubble_constant           = 0.695


Opening snapshot /Volumes/FoggieVida/halo_008508/ludicrous/nref13c_nref9f.enhance/DD2509/DD2509
get_refine_box: using this location:        col1          col2     col3     col4     col5     col6     col7   col8
------------------ -------- -------- -------- -------- -------- -------- ----
0.0021111231691204 0.488873 0.470349 0.508524 0.490873 0.472349 0.510524    9


Parsing Hierarchy : 100%|█████████████████████████████████████████| 14472/14472 [00:00<00:00, 32146.78it/s]
yt : [INFO     ] 2024-11-15 09:12:49,094 Gathering a field list (this may take a moment.)


This halo_c_v file doesn't exist, calculating halo center...
get_halo_center: code_length code_velocity
get_halo_center: obtained the spherical region
get_halo_center: extracted the DM density
get_halo_center: we have obtained the preliminary center
got the velocities
get_halo_center: located the main halo at: [0.48985743522644043, 0.47122740745544434, 0.5095312595367432] [unyt_quantity(0.001008, 'code_velocity'), unyt_quantity(-0.00177564, 'code_velocity'), unyt_quantity(0.0011421, 'code_velocity')]
filtering young_stars particles...
filtering young_stars3 particles...
filtering young_stars8 particles...
filtering old_stars particles...
filtering stars particles...
filtering dm particles...
using particle type  young_stars  to derive angular momentum
found angular momentum vector


In [12]:
load_and_calculate(snap, ions) #other options: filter_type='inflow_outflow' or filter_type=None ,or filter_type='temperature', filter_value=1e4


yt : [INFO     ] 2024-11-15 09:13:18,737 xlim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:13:18,737 ylim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:13:18,738 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-15 09:13:18,739 Making a fixed resolution buffer of (('gas', 'Emission_CIV_1548')) 1460 by 1460


z=0.0
CIV


yt : [INFO     ] 2024-11-15 09:16:09,271 Saving plot /Users/vidasaeedzadeh/Projects/foggie_outputs/ions_halo_008508/ludicrous/nref13c_nref9f.enhance/FRBs/DD2509_CIV_emission_map_edge-on_all.png
yt : [INFO     ] 2024-11-15 09:16:09,456 xlim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:16:09,456 ylim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:16:09,456 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-15 09:16:09,457 Making a fixed resolution buffer of (('gas', 'Emission_CIV_1548')) 1460 by 1460
yt : [INFO     ] 2024-11-15 09:18:24,027 Saving plot /Users/vidasaeedzadeh/Projects/foggie_outputs/ions_halo_008508/ludicrous/nref13c_nref9f.enhance/FRBs/DD2509_CIV_emission_map_face-on_all.png
yt : [INFO     ] 2024-11-15 09:18:24,205 xlim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:18:24,205 ylim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:18:24,205 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-15 09:18:24,206 Making a fixed resolution buffer of (('gas', 'Em

OVI


yt : [INFO     ] 2024-11-15 09:21:08,005 Saving plot /Users/vidasaeedzadeh/Projects/foggie_outputs/ions_halo_008508/ludicrous/nref13c_nref9f.enhance/FRBs/DD2509_OVI_emission_map_edge-on_all.png
yt : [INFO     ] 2024-11-15 09:21:08,152 xlim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:21:08,153 ylim = -0.000348 0.000348
yt : [INFO     ] 2024-11-15 09:21:08,153 zlim = -0.500000 0.500000
yt : [INFO     ] 2024-11-15 09:21:08,153 Making a fixed resolution buffer of (('gas', 'Emission_OVI')) 1460 by 1460
yt : [INFO     ] 2024-11-15 09:23:52,696 Saving plot /Users/vidasaeedzadeh/Projects/foggie_outputs/ions_halo_008508/ludicrous/nref13c_nref9f.enhance/FRBs/DD2509_OVI_emission_map_face-on_all.png


finished
